In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [13]:
with tf.Graph().as_default():
    with tf.name_scope("hyper_parameters"):
        keep_prob = tf.placeholder(tf.float32)
    with tf.name_scope("data"):
        xs = tf.placeholder(shape=[None, 784], dtype=tf.float32, name="xs")
        ys = tf.placeholder(shape=[None, 10], dtype=tf.float32, name="ys")
        xs_image = tf.reshape(xs, shape=[-1, 28, 28, 1], name="xs_image")
    with tf.name_scope("conv1"):
        wc1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=0.1), dtype=tf.float32, name="wc1")
        bc1 = tf.Variable(tf.constant(0.1, shape=[32]), dtype=tf.float32, name="bc1")
        conv1 = tf.nn.relu(tf.nn.conv2d(xs_image, wc1, strides=[1, 1, 1, 1], padding="SAME")+bc1, name="conv1")
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name="pool1")
    with tf.name_scope("conv2"):
        wc2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=0.1), dtype=tf.float32, name="wc2")
        bc2 = tf.Variable(tf.constant(0.1, shape=[64]), dtype=tf.float32, name="bc2")
        conv2 = tf.nn.relu(tf.nn.conv2d(pool1, wc2, strides=[1, 1, 1, 1], padding="SAME")+bc2, name="conv2")
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name="pool2")
        pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    with tf.name_scope("l1"):
        wl1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024], stddev=0.1), dtype=tf.float32, name="wl1")
        bl1 = tf.Variable(tf.constant(0.1, shape=[1024]), dtype=tf.float32, name="bl1")
        l1 = tf.nn.relu(tf.matmul(pool2_flat, wl1)+bl1, name="l1")
        drop1 = tf.nn.dropout(l1, keep_prob=keep_prob)
    with tf.name_scope("l2"):
        wl2 = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=0.1), dtype=tf.float32, name="wl2")
        bl2 = tf.Variable(tf.constant(0.1, shape=[10]), dtype=tf.float32, name="bl2")
        y_pred = tf.nn.softmax(tf.matmul(drop1, wl2)+bl2, name="y_pred")
    loss = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(y_pred), reduction_indices=[1]), name="loss")
    optimizer = tf.train.AdamOptimizer(1e-4)
    train = optimizer.minimize(loss)
    correct_prediction = tf.equal(tf.argmax(ys, 1), tf.argmax(y_pred, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    saver = tf.train.Saver()
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("mnist_cnn/logs/", sess.graph)
        sess.run(tf.global_variables_initializer())
        for step in range(501):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            sess.run(train, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5})
            if step%100 == 0:
                print("step: ", step)
                print("loss: ", sess.run(loss, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5}))
                print("accuracy: ", sess.run(accuracy, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5}))
                print("#"*50)
        saver.save(sess, "mnist_cnn/network/cnn.ckpt")

step:  0
loss:  12.509411
accuracy:  0.09
##################################################
step:  100
loss:  1.0996362
accuracy:  0.7
##################################################
step:  200
loss:  0.46764618
accuracy:  0.87
##################################################
step:  300
loss:  0.38845706
accuracy:  0.83
##################################################
step:  400
loss:  0.27107683
accuracy:  0.92
##################################################
step:  500
loss:  0.17706455
accuracy:  0.93
##################################################


In [5]:
with tf.Graph().as_default():
    def weight_variable(shape):
        initial = tf.truncated_normal(shape=shape, stddev=0.1)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    with tf.name_scope("data"):
        xs = tf.placeholder(tf.float32, [None, 784], name="xs")
        ys = tf.placeholder(tf.float32, [None, 10], name="ys")
        xs_image = tf.reshape(xs, [-1, 28, 28, 1], name="xs_image")
    with tf.name_scope("conv1"):
        wc1 = weight_variable([5, 5, 1, 32])
        bc1 = bias_variable([32])
        conv1 = tf.nn.relu(conv2d(xs_image, wc1)+bc1)
        pool1 = max_pool_2x2(conv1)
    with tf.name_scope("conv2"):
        wc2 = weight_variable([5, 5, 32, 64])
        bc2 = bias_variable([64])
        conv2 = tf.nn.relu(conv2d(pool1, wc2)+bc2)
        pool2 = max_pool_2x2(conv2)
        pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    keep_prob = tf.placeholder(tf.float32)
    with tf.name_scope("l1"):
        wl1 = weight_variable([7*7*64, 1024])
        bl1 = weight_variable([1024])
        l1 = tf.nn.relu(tf.matmul(pool2_flat, wl1)+bl1)
        drop1 = tf.nn.dropout(l1, keep_prob=keep_prob)
    with tf.name_scope("l2"):
        wl2 = weight_variable([1024, 10])
        bl2 = weight_variable([10])
        y_pred = tf.nn.softmax(tf.matmul(drop1, wl2)+bl2)
    loss = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(y_pred), reduction_indices=[1]))
    optimizer = tf.train.AdamOptimizer(1e-4)
    train = optimizer.minimize(loss)
    correct_prediction = tf.equal(tf.argmax(ys, 1), tf.argmax(y_pred, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(1001):
            batch = mnist.train.next_batch(50)
            sess.run(train, feed_dict={xs:batch[0], ys:batch[1], keep_prob:0.5})
            if step%100 == 0:
                print("step: ", step)
                print("loss: ", sess.run(loss, feed_dict={xs:batch[0], ys:batch[1], keep_prob:0.5}))
                print("accuracy: ", sess.run(accuracy, feed_dict={xs:batch[0], ys:batch[1], keep_prob:0.5}))

step:  0
loss:  9.715934
accuracy:  0.1
step:  100
loss:  1.141737
accuracy:  0.46
step:  200
loss:  0.34124348
accuracy:  0.9
step:  300
loss:  0.6595492
accuracy:  0.82
step:  400
loss:  0.43005794
accuracy:  0.92
step:  500
loss:  0.17584221
accuracy:  0.94


KeyboardInterrupt: 